In [1]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
#import tensorflow_datasets as tfds
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn import preprocessing , neighbors ,svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [61]:
xss = pd.read_csv('Data/XSS_dataset.csv')
xss = xss.fillna(0)
xss.Sentence=xss.Sentence.astype(str)
#xss.SentimentText=xss.Label.astype(str)

In [62]:
def leng(df,col,len_col):
  for i in range(len(df)):
    cl = df[col][i]
    length = len(str(cl))
    df[len_col][i] = length
  return df

In [63]:
xss['Length']=0
xss = leng(xss, 'Sentence','Length')

C:\Users\hamza\anaconda3\envs\hello-tf\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [64]:
xss.head()


,Sentence,Label,Length
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0.0,557
1,"<tt onmouseover=""alert(1)"">test</tt>",1.0,36
2,"\t </span> <span class=""reference-text"">Steeri...",0.0,233
3,"\t </span> <span class=""reference-text""><cite ...",0.0,395
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0.0,422


In [65]:
def cal_puncndop(df,col,punop_col,l):
    special_characters = ['!', '#','$','%', '.', ':', '&','@','[',']',' ',']','_', '<', '>', '=', '"', '/', '(', ')']
    for i in range(len(df[col])):
        count = 0.0
        query = df[col][i].lower()
        #print(query)
        for q in special_characters:
          if q in query:
            count+=1
        df[punop_col][i]=count
    return df 

In [66]:
xss['punctuation'] = 0
#df['operator'] = 0
xss=cal_puncndop(xss,'Sentence','punctuation',['!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?","[","]",")","("])
#df=cal_puncndop(df,'Query','operator',["<",">", "<=", ">=", "=", "==", "!=", "<<", ">>", "|", "&", "-", "+", "%", "^", "*"]) #should * be included becoz count(*) !!!
xss.head()

C:\Users\hamza\anaconda3\envs\hello-tf\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Sentence,Label,Length,punctuation
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0.0,557,10
1,"<tt onmouseover=""alert(1)"">test</tt>",1.0,36,8
2,"\t </span> <span class=""reference-text"">Steeri...",0.0,233,9
3,"\t </span> <span class=""reference-text""><cite ...",0.0,395,10
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0.0,422,12


In [80]:
def cal_keyword(df,col,key_col,l):
  special_characters = ['!', '#','$','%', '.', ':', '&','@','[',']',' ',']','_', '<', '>', '=', '"', '/', '(', ')']
  for i in range(len(df[col])):
    count = 1.0
    query = df[col][i].lower()
    #print(query)
    for q in special_characters:
      query = query.replace(q,' ')
    #print(query+"\n ............................")
    words = query.split()
    #print(words)
    for word in words:
      if word in l:
        count = count+1
    df[key_col][i] = count
  return df

In [81]:
xss['keyword'] = 0
xss = cal_keyword(xss, 'Sentence', 'keyword', ['alert','onload', 'onmouseover', 'vbscript', 'script', "src","onactive","onafterscriptexecute","onanimationcancel","onanimationend","onanimationiteration","onanimationstart","onbeforeactive","onbeforedeactivate","onbeforeprint","onbeforescriptexecute","onbeforeunload",
                                              "onbegin","onblur","onbounce","oncanplay","oncanplaythrouhgh","oncuechange","ondeactivate","ondurationchange","onend","onended","a","href"," onmouseleave"])
xss.head()

C:\Users\hamza\anaconda3\envs\hello-tf\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Sentence,Label,Length,punctuation,keyword
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0,557,10,8
1,"<tt onmouseover=""alert(1)"">test</tt>",1,36,8,3
2,"\t </span> <span class=""reference-text"">Steeri...",0,233,9,5
3,"\t </span> <span class=""reference-text""><cite ...",0,395,10,4
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0,422,12,10


In [82]:
xss.to_csv('Data/XSS_dataset_processed.csv')

In [83]:
def scale_data(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('float')
        encoder = preprocessing.StandardScaler()
        df[column] = encoder.fit_transform(df[column].values.reshape(-1,1))
        print("The ",column, "is encoded")
    return(df)
  
def encode_categorical(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
#         print(encoded_list)
#         print(len(encoded_list))
        encoded_series = pd.Series(encoded_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
    return(df)

xss = encode_categorical(xss,['Label'])

The  Label is encoded 


In [84]:
X = np.array(xss.drop(labels=['Label', 'Sentence'],axis=1)).reshape(len(xss),1,3)
print(X)
# y_train = np.asarray(xss.drop(labels=['Label', 'Sentence'],axis=1)).astype('float32').reshape((-1,1))
# print(X)
y = np.array(xss['Label'])
print(y)

[[[557  10   8]]

 [[ 36   8   3]]

 [[233   9   5]]

 ...

 [[  6   4   1]]

 [[ 38   7   2]]

 [[ 62   8   2]]]
[0 1 0 ... 1 1 0]


In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [91]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))


In [92]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [93]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, activation='relu', input_shape= (None,3)))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, None, 20)          80        
_________________________________________________________________
dense_25 (Dense)             (None, None, 16)          336       
_________________________________________________________________
dense_26 (Dense)             (None, None, 1)           17        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [94]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [95]:
history = model.fit(train_dataset,
                    epochs=20,
                    verbose=1)

Epoch 1/20
162/162 [==============================] - 1s 1ms/step - loss: 0.6924 - accuracy: 0.4572
Epoch 2/20
162/162 [==============================] - 0s 1ms/step - loss: 0.6821 - accuracy: 0.4522
Epoch 3/20
162/162 [==============================] - 0s 2ms/step - loss: 0.6795 - accuracy: 0.4819
Epoch 4/20
162/162 [==============================] - 0s 1ms/step - loss: 0.6765 - accuracy: 0.5297
Epoch 5/20
162/162 [==============================] - 0s 1ms/step - loss: 0.6749 - accuracy: 0.5464
Epoch 6/20
162/162 [==============================] - 0s 1ms/step - loss: 0.6741 - accuracy: 0.5509
Epoch 7/20
162/162 [==============================] - 0s 2ms/step - loss: 0.6738 - accuracy: 0.5541
Epoch 8/20
162/162 [==============================] - 0s 1ms/step - loss: 0.6735 - accuracy: 0.5578
Epoch 9/20
162/162 [==============================] - 0s 1ms/step - loss: 0.6728 - accuracy: 0.5624
Epoch 10/20
162/162 [==============================] - 0s 1ms/step - loss: 0.6727 - accuracy: 0.5586

In [96]:
model.evaluate(test_dataset)

41/41 [==============================] - 0s 1ms/step - loss: 0.6667 - accuracy: 0.5899


[0.6667225360870361, 0.5898528099060059]